ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [25]:
pip install numpy==1.23.5 scikit-learn==1.4.2  joblib pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
!python3 -m pip install pip==24.0



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\vhgar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!python --version

Python 3.11.9


In [4]:
!pip install azureml-sdk

In [1]:

import json

#GETTING MY ID:
id = open('../id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]
#my_id

In [2]:
## STEP 1: Set up the workspace.
from azureml.core import Workspace

ws = Workspace.create(name="workspace_vf",
                      subscription_id = my_id,
                      resource_group = "resource_group_bank_vf",
                      location = "centralindia")



Deploying StorageAccount with name workspacstorage543fb21cd.
Deploying KeyVault with name workspackeyvaultec4956a1.
Deploying AppInsights with name workspacinsightsc7e9b6d8.
Deployed AppInsights with name workspacinsightsc7e9b6d8. Took 7.5 seconds.
Deployed StorageAccount with name workspacstorage543fb21cd. Took 29.57 seconds.
Deployed KeyVault with name workspackeyvaultec4956a1. Took 27.48 seconds.
Deploying Workspace with name workspace_vf.
Deployed Workspace with name workspace_vf. Took 41.08 seconds.


In [3]:
from azureml.core.model import Model

mname = "model3"
registered_model = Model.register(model_path="../Model/best_model3.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model3


In [4]:
from azureml.core.model import Model

mname_scaler = "model_scaler3"
registered_scaler = Model.register(model_path="../Model/scaler2.pkl",
                                  model_name=mname_scaler,
                                  workspace=ws)

Registering model model_scaler3


In [5]:
scorepy = """
import json 
import joblib
import pandas as pd
import numpy as np
import traceback
import sklearn
from azureml.core.model import Model

def embbed(d):
    try:
        required_columns =  [' Net Income to Total Assets', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax']
        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

def init():
    global model, scaler

    try:
        print("[INFO] Loading model and scaler...")
        print(f"[INFO] NumPy version: {np.__version__}")
        print(f"[INFO] Scikit-learn version: {sklearn.__version__}")

        # Reemplaza 'model_name_here' por el nombre real de tu modelo registrado en Azure ML.
        model_path = Model.get_model_path('model3')
        scaler_path = Model.get_model_path('model_scaler3')

        print(f"[INFO] Loading model from {model_path}")
        model = joblib.load(model_path)
        print(f"[INFO] Loaded model from {model_path}!!")

        print(f"[INFO] Loading scaler from {scaler_path}")
        scaler = joblib.load(scaler_path)
        print(f"[INFO] Loaded scaler from {scaler_path}!!")

        print("[INFO] Model and scaler loaded successfully.")

    except Exception as e:
        print(f"[ERROR] Failed to load model or scaler: {e}")
        traceback.print_exc()
        model, scaler = None, None  # Aseguramos que sean None si falla la carga

def run(raw_data):
    global model, scaler

    try:
        if model is None or scaler is None:
            raise RuntimeError("[ERROR] Model or scaler not initialized. Check logs for errors.")

        data_dict = json.loads(raw_data)
        if "data" not in data_dict:
            raise ValueError("[ERROR] Input JSON must contain a 'data' key.")

        data = data_dict["data"]
        if not isinstance(data, list):
            raise ValueError("[ERROR] Expected 'data' to be a list of feature dictionaries.")

        df = pd.DataFrame(data)

        print("[INFO] Received data shape:", df.shape)
        print("[INFO] Data columns:", df.columns.tolist())

        embedded_data = embbed(df)
        if embedded_data is None:
            raise ValueError("[ERROR] Data embedding failed due to missing columns.")

        # Verificar si hay valores NaN antes de transformar
        if embedded_data.isnull().values.any():
            raise ValueError("[ERROR] Input data contains NaN values. Please clean your input.")

        scaled_data = scaler.transform(embedded_data)
        print("[INFO] Scaled data shape:", scaled_data.shape)

        result_finals = model.predict(scaled_data)
        print("[INFO] Model prediction output:", result_finals)

        return json.dumps({"predictions": result_finals.tolist()})

    except json.JSONDecodeError as e:
        error_msg = f"[ERROR] Invalid JSON input: {e}"
    except ValueError as e:
        error_msg = str(e)
    except Exception as e:
        error_msg = f"[ERROR] Unexpected error in run(): {e} {traceback.format_exc()}"

    print(error_msg)
    return json.dumps({"error": error_msg})


"""

In [6]:


# Guardar el script en un archivo
file_score = open("score.py", "w")
# Guardar el script en un archivo con codificación UTF-8
with open("score.py", "w", encoding="utf-8") as file_score:
    file_score.write(scorepy)

file_score.close()


In [7]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-for-bank-1-3")


from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy==1.23.5','scikit-learn=1.4.2', 'joblib', 'pandas']
)


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model, registered_scaler],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\vhgar\AppData\Local\Temp\ipykernel_58428\3279380664.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [11]:
virtual_env

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "AZUREML_ENTRY_SCRIPT": "score.py",
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "env-for-bank-1-3",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": 

In [8]:
import joblib
import json

model = joblib.load("../Model/best_model4.pkl")
scaler = joblib.load("../Model/scaler2.pkl")

import pandas as pd

def embbed(d):
    try:
        required_columns =  [' Net Income to Total Assets', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax']
        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

data = pd.read_csv("../prueba.csv")
# Formatear correctamente para la API
data_dict = data.to_dict(orient='records')  # Cambia 'list' por 'records'
raw_data = json.dumps({"data": data_dict})


if model is None or scaler is None:
    raise RuntimeError("[ERROR] Model or scaler not initialized. Check logs for errors.")

data_dict = json.loads(raw_data)
if "data" not in data_dict:
    raise ValueError("[ERROR] Input JSON must contain a 'data' key.")

data = data_dict["data"]
if not isinstance(data, list):
    raise ValueError("[ERROR] Expected 'data' to be a list of feature dictionaries.")

df = pd.DataFrame(data)

print("[INFO] Received data shape:", df.shape)
print("[INFO] Data columns:", df.columns.tolist())

embedded_data = embbed(df)
if embedded_data is None:
    raise ValueError("[ERROR] Data embedding failed due to missing columns.")

# Verificar si hay valores NaN antes de transformar
if embedded_data.isnull().values.any():
    raise ValueError("[ERROR] Input data contains NaN values. Please clean your input.")

scaled_data = scaler.transform(embedded_data)
print("[INFO] Scaled data shape:", scaled_data.shape)

result_finals = model.predict(scaled_data)
print("[INFO] Model prediction output:", result_finals)

[INFO] Received data shape: (10, 97)
[INFO] Data columns: ['Unnamed: 0', 'Bankrupt?', ' ROA(C) before interest and depreciation before interest', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax', ' Operating Gross Margin', ' Realized Sales Gross Margin', ' Operating Profit Rate', ' Pre-tax net Interest Rate', ' After-tax net Interest Rate', ' Non-industry income and expenditure/revenue', ' Continuous interest rate (after tax)', ' Operating Expense Rate', ' Research and development expense rate', ' Cash flow rate', ' Interest-bearing debt interest rate', ' Tax rate (A)', ' Net Value Per Share (B)', ' Net Value Per Share (A)', ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons', ' Cash Flow Per Share', ' Revenue Per Share (Yuan ¥)', ' Operating Profit Per Share (Yuan ¥)', ' Per Share Net profit before tax (Yuan ¥)', ' Realized Sales Gross Profit Growth Rate', ' Operating Profit Growth Rate', ' After-tax Net Profit Growth R

In [9]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"


In [13]:
models = Model.list(workspace=ws)
for model in models:
    print(model.name, model.version)


model_scaler 1
model 1


In [52]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_9c4920aae0e7dc650e488f37eb5f261f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_9c4920aae0e7dc650e488f37eb5f261f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_9c4920aae0e7dc650e488f37eb5f261f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-05T02:16:22,548495888+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_9c4920aae0e7dc650e488f37eb5f261f/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-05T02:16:22,558763079+00:00 - gunicorn/run 
2025-04-05T02:16:22,560292502+00:00 | gunicorn/run | 
2025-04-05T02:16:22,563316522+00:00 | gunicorn/run | ###############################################
2025-04-05T02:16:22,567274224+00:00 - nginx/run 
2025-04-05T02:16:22,567520604+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-05T02:16:22,568872155+00:00 | gunico